[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DroneBlocks/dexi_yolo_training/blob/main/DEXI_YOLO_Training_Google_Colab.ipynb)

# DEXI YOLO Training Tutorial - Google Colab

This notebook walks you through the complete process of training a custom YOLO object detection model for drone detection using Google Colab's free GPU. We'll be working with 6 classes: bird, dog, cat, motorcycle, car, and truck.

## 🚀 Quick Setup:
1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → **T4 GPU**
2. **Run first cell** → Automatically clones repo and installs everything
3. **Run all cells** → Complete YOLO training pipeline in ~20 minutes!

## 📋 Table of Contents
1. [Colab Setup & Repository Clone](#colab-setup)
2. [Environment Setup](#environment-setup)
3. [Dataset Exploration](#dataset-exploration) 
4. [Data Augmentation](#data-augmentation)
5. [YOLO Training](#yolo-training)
6. [Results Analysis](#results-analysis)
7. [Model Testing](#model-testing)
8. [ONNX Conversion](#onnx-conversion)
9. [Download Models](#download-models)

---

## 1. Colab Setup & Repository Clone

This cell automatically detects if we're running in Google Colab and sets up everything needed:
- Clones the DEXI YOLO Training repository
- Installs all required dependencies
- Pre-downloads the YOLO model

**⚠️ Important: Enable GPU before running!**
- Go to: **Runtime → Change runtime type → Hardware accelerator → T4 GPU**

In [ ]:
# 🌟 GOOGLE COLAB SETUP - Clone Repository & Install Dependencies
import os
from pathlib import Path
import sys

# Check if we're in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌟 Running in Google Colab")
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

if IN_COLAB:
    # Clone the repository if it doesn't exist
    if not Path('dexi_yolo_training').exists():
        print("📥 Cloning DEXI YOLO Training repository...")
        !git clone https://github.com/DroneBlocks/dexi_yolo_training.git
        print("✅ Repository cloned successfully!")
    else:
        print("📁 Repository already exists")
    
    # Change to repo directory
    os.chdir('dexi_yolo_training')
    print(f"📂 Current directory: {os.getcwd()}")
    
    # Install requirements
    print("🔧 Installing requirements...")
    !pip install -r requirements.txt -q
    
    # Pre-download YOLO model to speed up later steps
    print("📦 Pre-downloading YOLO model...")
    from ultralytics import YOLO
    YOLO('yolov8n.pt')  # This downloads and caches the model
    
    print("\n🎉 Colab setup complete! Ready to train YOLO model.")
    print("💡 Make sure GPU is enabled: Runtime → Change runtime type → T4 GPU")
    print("🚀 Expected training time with GPU: 15-25 minutes")
    
else:
    print("💻 Local setup detected - skipping Colab-specific steps")
    print("📝 Make sure you have activated your virtual environment")
    print("⚡ Run: pip install -r requirements.txt")

## 2. Environment Setup

Let's check our hardware and import all necessary libraries. This will detect whether we have GPU acceleration available.

In [ ]:
# Import required libraries and check hardware acceleration
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
from ultralytics import YOLO
import torch
from IPython.display import Image, display
import pandas as pd
from glob import glob
import glob as globmodule

# Set matplotlib style for better plots
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)

print(f"🔧 System Information:")
print(f"   PyTorch version: {torch.__version__}")

# Enhanced device detection for Colab
if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🚀 GPU: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    
    # Colab-specific GPU info
    try:
        import google.colab
        print(f"⚡ Google Colab GPU acceleration enabled!")
        print(f"   Expected training time: 15-25 minutes")
        print(f"   Expected augmentation time: 2-3 minutes")
    except ImportError:
        print(f"🖥️  Local CUDA GPU detected")
        
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = 'mps' 
    print(f"⚡ Apple Silicon MPS acceleration!")
    print(f"   Expected training time: 30-45 minutes")
    torch.mps.empty_cache()
else:
    device = 'cpu'
    print(f"💻 Using CPU")
    
    # Colab warning if no GPU
    try:
        import google.colab
        print("⚠️ WARNING: GPU not enabled in Colab!")
        print("💡 Enable GPU: Runtime → Change runtime type → Hardware accelerator → T4 GPU")
        print("   Then: Runtime → Restart runtime and run all cells again")
    except ImportError:
        print(f"   Expected training time: 2-4 hours")

print(f"\n✅ Selected device: {device}")

## 3. Dataset Exploration

Let's explore our dataset structure and examine the original 6 images that will be used for augmentation.

In [ ]:
# Find and display our original images
original_images_path = Path('train/original_image')
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']

# Find all image files
all_images = []
for ext in image_extensions:
    all_images.extend(original_images_path.glob(ext))
    all_images.extend(original_images_path.glob(ext.upper()))

print(f"📸 Found {len(all_images)} original images in the dataset")
print(f"💡 These will be augmented to create 900 training images (150 per class)")

# Display original images
if all_images:
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Original Images for YOLO Training', fontsize=16, fontweight='bold')
    
    for idx, img_path in enumerate(all_images[:6]):
        if idx >= 6:
            break
        
        row = idx // 3
        col = idx % 3
        
        # Load and display image
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[row, col].imshow(img_rgb)
        axes[row, col].set_title(f"{img_path.stem.upper()}\n{img.shape[1]}x{img.shape[0]}px", fontweight='bold')
        axes[row, col].axis('off')
    
    # Hide unused subplots
    for idx in range(len(all_images), 6):
        row = idx // 3
        col = idx % 3
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n📊 Original Images Summary:")
    for img_path in all_images[:6]:
        img = cv2.imread(str(img_path))
        size_mb = img_path.stat().st_size / (1024*1024)
        print(f"  {img_path.name}: {img.shape[1]}x{img.shape[0]} ({size_mb:.1f} MB)")
        
else:
    print("❌ No images found in the train/original_image directory")
    print("🔍 Make sure the repository was cloned correctly")

## 4. Data Augmentation

Now we'll use our custom augmentation script to create 150 variations of each base image (900 total). This is crucial for training a robust YOLO model as it helps the model generalize better to different conditions.

**⏱️ Expected time: 2-3 minutes with GPU, 5-7 minutes with CPU**

In [ ]:
# Let's examine our augmentation script first
print("🔧 Data Augmentation Pipeline Overview:")
print("")
print("Our augmentation script applies the following transformations:")
print("• 🔄 Rotation: 0-360 degrees (random)")
print("• 📏 Scaling: 0.25x to 1.3x (random)")
print("• ☀️ Brightness: -30 to +30 (random)")
print("• 🌈 Contrast: 0.7x to 1.3x (random)")
print("• 📻 Noise: Added 20% of the time")
print("• 🌫️ Blur: Applied 15% of the time")
print("")
print("Each transformation creates realistic variations that help the model")
print("learn to detect objects under different lighting, weather, and camera conditions.")
print("")
print("💡 This technique dramatically improves model robustness with minimal data!")

In [ ]:
# Set augmentation parameters
AUGMENTATIONS_PER_IMAGE = 150  # Creates 900 total images (6 classes × 150)
INPUT_DIR = "train/original_image"     # Directory with original images
OUTPUT_DIR = "train"          # Output directory for augmented dataset

print(f"⚙️ Augmentation Configuration:")
print(f"   Input directory: {INPUT_DIR}")
print(f"   Output directory: {OUTPUT_DIR}")
print(f"   Augmentations per image: {AUGMENTATIONS_PER_IMAGE}")
print(f"   Expected total images: {len(all_images) * AUGMENTATIONS_PER_IMAGE}")
print(f"")
print(f"📊 Training Dataset Breakdown:")
for img_path in all_images[:6]:
    class_name = img_path.stem
    print(f"   {class_name}: {AUGMENTATIONS_PER_IMAGE} augmented images")

In [ ]:
# Run the augmentation process
print("🚀 Starting data augmentation...")

# Time estimation based on environment
if device == 'cuda':
    print("⚡ GPU acceleration detected - estimated time: 2-3 minutes")
elif device == 'mps':
    print("🍎 Apple Silicon detected - estimated time: 3-4 minutes")
else:
    print("💻 CPU processing - estimated time: 5-7 minutes")

print("📈 Progress will be shown below...")
print("")

# Import and use our augmentation class
from augment_dataset import YOLODatasetAugmenter

# Create augmenter instance
augmenter = YOLODatasetAugmenter(INPUT_DIR, OUTPUT_DIR)

# Run augmentation with progress tracking
import time
start_time = time.time()

augmenter.augment_all_images(AUGMENTATIONS_PER_IMAGE)

end_time = time.time()
duration = end_time - start_time

print(f"\n✅ Data augmentation completed in {duration/60:.1f} minutes!")
print(f"🎯 Ready for YOLO training with {len(all_images) * AUGMENTATIONS_PER_IMAGE} images")

In [ ]:
# Verify augmentation results
train_images_dir = Path('train/images')
train_labels_dir = Path('train/labels')

# Count generated files
augmented_images = list(train_images_dir.glob('*_[0-9][0-9][0-9].jpg'))
augmented_labels = list(train_labels_dir.glob('*_[0-9][0-9][0-9].txt'))
original_images = [f for f in train_images_dir.glob('*.jpg') if not f.name.endswith(('_001.jpg', '_002.jpg', '_003.jpg'))]

print(f"📊 Augmentation Results Summary:")
print(f"   Original base images: {len(original_images)}")
print(f"   Generated images: {len(augmented_images)}")
print(f"   Total training images: {len(list(train_images_dir.glob('*.jpg')))}")
print(f"   Label files created: {len(augmented_labels)}")
print(f"")
print(f"🎯 Images per class (including originals):")
for class_name in ['bird', 'dog', 'cat', 'motorcycle', 'car', 'truck']:
    class_images = len(list(train_images_dir.glob(f'{class_name}*.jpg')))
    print(f"   {class_name.title()}: {class_images} images")

# Verify dataset balance
total_images = len(list(train_images_dir.glob('*.jpg')))
if total_images == 906:  # 6 original + 900 augmented
    print(f"\n✅ Dataset verification passed! Ready for training.")
else:
    print(f"\n⚠️ Expected 906 images, found {total_images}. Check augmentation process.")

In [ ]:
# Display a few examples of augmented images
print("🖼️ Sample Augmented Images (Showing Transformations):")

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Augmented Training Images - Variety of Transformations', fontsize=16, fontweight='bold')

# Show examples from each class
classes = ['bird', 'dog', 'cat', 'motorcycle', 'car', 'truck']
sample_count = 0

for class_idx, class_name in enumerate(classes):
    # Get first 2 augmented images of each class
    class_images = list(train_images_dir.glob(f'{class_name}_*.jpg'))[:2]
    
    for img_idx, img_path in enumerate(class_images):
        if sample_count >= 12:  # 3x4 grid
            break
            
        row = sample_count // 4
        col = sample_count % 4
        
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        axes[row, col].imshow(img_rgb)
        axes[row, col].set_title(f"{class_name.title()}\n{img_path.name}", fontweight='bold', fontsize=10)
        axes[row, col].axis('off')
        
        sample_count += 1
    
    if sample_count >= 12:
        break

# Hide unused subplots
for idx in range(sample_count, 12):
    row = idx // 4
    col = idx % 4
    axes[row, col].axis('off')

plt.tight_layout()
plt.show()

print("\n🎨 Notice the variety in rotations, brightness, and transformations!")
print("🧠 This diversity helps the model generalize to real-world conditions.")

## 5. YOLO Training

Now that we have our augmented dataset ready, let's train our YOLO model! We'll use the YOLOv8 nano architecture, which is perfect for edge deployment while still achieving excellent accuracy.

**⏱️ Expected time: 15-25 minutes with T4 GPU, 30-45 minutes with local GPU**

In [ ]:
# Training configuration optimized for Colab
TRAINING_CONFIG = {
    'model_size': 'n',        # Nano: Fast, efficient, perfect for Pi deployment
    'epochs': 100,            # Number of training epochs
    'imgsz': 640,             # Image size for training
    'batch_size': 16 if device == 'cuda' else 8,  # Adjust for GPU memory
    'device': device,         # Device determined earlier
}

print("🎯 Training Configuration:")
for key, value in TRAINING_CONFIG.items():
    print(f"   {key}: {value}")
    
print("\n💡 YOLOv8 Model Size Guide:")
print("   • 'n' (nano): ⚡ Fastest, smallest, perfect for Pi/mobile deployment")
print("   • 's' (small): 🔄 Good balance of speed and accuracy")
print("   • 'm' (medium): 🎯 Better accuracy, moderate speed")
print("   • 'l' (large): 🏆 High accuracy, slower inference")
print("   • 'x' (extra-large): 🥇 Highest accuracy, slowest inference")

print(f"\n⚡ Performance Expectations ({device.upper()}):")
if device == 'cuda':
    print("   Training time: ~15-25 minutes")
    print("   Expected mAP@0.5: 99%+ (excellent!)")
elif device == 'mps':
    print("   Training time: ~30-45 minutes") 
    print("   Expected mAP@0.5: 99%+ (excellent!)")
else:
    print("   Training time: 2-4 hours")
    print("   Consider enabling GPU for much faster training")

In [ ]:
# Initialize the YOLO model
model_name = f"yolov8{TRAINING_CONFIG['model_size']}.pt"
print(f"🤖 Loading {model_name} model...")

# Load pre-trained YOLO model
model = YOLO(model_name)

print(f"\n✅ Model loaded successfully!")
print(f"   Model: YOLOv8{TRAINING_CONFIG['model_size']}")
print(f"   Parameters: {sum(p.numel() for p in model.model.parameters()):,}")
print(f"   Model size: {os.path.getsize(model_name) / (1024*1024):.1f} MB")
print(f"   Perfect for Raspberry Pi deployment! 🥧")

In [ ]:
# Start training with progress monitoring
print("🚀 Starting YOLO training...")
print("📊 You'll see training progress, loss curves, and validation metrics below.")
print("💾 Training logs and checkpoints will be saved in 'runs/detect/drone_detection/'")
print("💡 YOLO will automatically split training data for validation (80% train, 20% val)")
print("")

# Time estimation
if device == 'cuda':
    print("⏱️  Estimated completion: 15-25 minutes")
    print("☕ Perfect time for a coffee break!")
elif device == 'mps':
    print("⏱️  Estimated completion: 30-45 minutes")
    print("🍵 Great time for tea and checking emails!")
else:
    print("⏱️  Estimated completion: 2-4 hours")
    print("💡 Consider enabling GPU: Runtime → Change runtime type → T4 GPU")

print("\n🎯 Training starting...")
print("=" * 60)

import time
training_start_time = time.time()

# Train the model with optimized parameters
results = model.train(
    data='dataset.yaml',
    epochs=TRAINING_CONFIG['epochs'],
    imgsz=TRAINING_CONFIG['imgsz'],
    batch=TRAINING_CONFIG['batch_size'],
    device=TRAINING_CONFIG['device'],
    project='runs/detect',
    name='drone_detection',
    save_period=20,      # Save checkpoint every 20 epochs (less frequent for Colab)
    patience=25,         # Early stopping patience
    
    # Automatic train/validation split (no separate val folder needed)
    fraction=0.8,        # 80% for training, 20% for validation
    
    # Augmentation settings (additional to our pre-generated augmentations)
    hsv_h=0.015,         # Hue augmentation
    hsv_s=0.7,           # Saturation augmentation  
    hsv_v=0.4,           # Value augmentation
    degrees=0,           # No additional rotation (we already did this)
    translate=0.1,       # Translation augmentation
    scale=0.1,           # Additional scale augmentation
    shear=0.1,           # Shear augmentation
    perspective=0.0,     # Perspective augmentation
    flipud=0.0,          # No vertical flip (objects have orientation)
    fliplr=0.0,          # No horizontal flip (for consistency)
    mosaic=0.8,          # Mosaic augmentation probability
    mixup=0.1,           # Mixup augmentation probability
    
    # Optimization settings
    optimizer='AdamW',   # Adam with weight decay
    lr0=0.01,            # Initial learning rate
    lrf=0.1,             # Final learning rate (lr0 * lrf)
    momentum=0.937,      # SGD momentum
    weight_decay=0.0005, # Weight decay for regularization
    warmup_epochs=3,     # Warm-up epochs
    warmup_momentum=0.8, # Warm-up momentum
    warmup_bias_lr=0.1,  # Warm-up bias learning rate
    
    # Loss function weights
    box=7.5,             # Box loss gain
    cls=0.5,             # Class loss gain  
    dfl=1.5,             # DFL loss gain
    verbose=True,        # Verbose output
)

training_end_time = time.time()
training_duration = training_end_time - training_start_time

print("=" * 60)
print(f"🎉 Training completed in {training_duration/60:.1f} minutes!")
print(f"🏆 Model ready for deployment!")

## 6. Results Analysis

Let's analyze the training results and visualize the model's performance with charts and metrics.

In [ ]:
# Find the latest training results directory
results_dirs = globmodule.glob('runs/detect/drone_detection*')
if results_dirs:
    results_dir = Path(max(results_dirs, key=os.path.getmtime))
    print(f"📂 Latest training results: {results_dir}")
    print(f"📁 Training artifacts:")
    for item in sorted(results_dir.iterdir()):
        if item.is_file():
            print(f"   📄 {item.name}")
        else:
            print(f"   📁 {item.name}/")
else:
    print("❌ Training results not found. Make sure training completed successfully.")
    results_dir = None

In [ ]:
# Display training curves
if results_dir and results_dir.exists():
    results_image_path = results_dir / 'results.png'
    if results_image_path.exists():
        print("📈 Training Results & Performance Curves:")
        print("These show how the model learned over time - loss should decrease, mAP should increase!")
        display(Image(str(results_image_path)))
        
        print("\n📊 How to read these charts:")
        print("• 📉 Loss curves (lower is better): Shows training progress")
        print("• 📈 mAP curves (higher is better): Shows detection accuracy")
        print("• 🎯 Precision/Recall: Shows detection quality")
        print("• ⚡ Look for: Decreasing loss, increasing mAP > 0.9")
    else:
        print("📈 Training curves not found. They should be available after training completes.")
else:
    print("❌ No results directory found.")

In [ ]:
# Display confusion matrix
if results_dir and results_dir.exists():
    confusion_matrix_path = results_dir / 'confusion_matrix.png'
    if confusion_matrix_path.exists():
        print("🎯 Confusion Matrix - Model Classification Accuracy:")
        print("This shows how well the model distinguishes between different classes.")
        print("Perfect diagonal = 100% accuracy for each class!")
        display(Image(str(confusion_matrix_path)))
        
        print("\n📊 How to read the confusion matrix:")
        print("• Diagonal (top-left to bottom-right): Correct predictions")
        print("• Off-diagonal: Misclassifications between classes")
        print("• Darker blue = higher values = better performance")
        print("• Goal: Dark diagonal, light off-diagonal areas")
    else:
        print("🎯 Confusion matrix not found.")
else:
    print("❌ No results directory found.")

In [ ]:
# Display validation batch results
if results_dir and results_dir.exists():
    val_batch_path = results_dir / 'val_batch0_labels.jpg'
    val_pred_path = results_dir / 'val_batch0_pred.jpg'
    
    if val_batch_path.exists():
        print("🔍 Validation Batch - Ground Truth Labels:")
        print("These are the 'correct answers' the model should learn.")
        display(Image(str(val_batch_path)))
    
    if val_pred_path.exists():
        print("\n🤖 Validation Batch - Model Predictions:")
        print("These are what the model actually predicted. Compare with ground truth above!")
        print("Good predictions = boxes in same locations with correct labels and high confidence.")
        display(Image(str(val_pred_path)))
        
        if val_batch_path.exists():
            print("\n💡 Comparison Tips:")
            print("• ✅ Bounding boxes should align well between ground truth and predictions")
            print("• ✅ Class labels should match (bird, car, dog, etc.)")
            print("• ✅ Confidence scores should be high (>0.5, ideally >0.8)")
            print("• 🎯 Perfect alignment = excellent model performance!")

    if not val_batch_path.exists() and not val_pred_path.exists():
        print("🔍 Validation images not found.")
else:
    print("❌ No results directory found.")

In [ ]:
# Load the trained model and run final validation
if results_dir and results_dir.exists():
    best_model_path = results_dir / 'weights' / 'best.pt'
    if best_model_path.exists():
        print(f"🏆 Loading best trained model: {best_model_path}")
        trained_model = YOLO(str(best_model_path))
        
        print(f"📊 Model file size: {best_model_path.stat().st_size / (1024*1024):.1f} MB")
        print(f"🥧 Perfect size for Raspberry Pi deployment!")
        
        # Run final validation
        print("\n🔬 Running final validation...")
        val_results = trained_model.val()
        
        # Print key metrics with explanations
        print("\n🎯 Final Model Performance Metrics:")
        print("=" * 50)
        if hasattr(val_results, 'box'):
            metrics = val_results.box
            map50 = metrics.map50
            map_5095 = metrics.map
            precision = metrics.mp
            recall = metrics.mr
            
            print(f"📈 mAP@0.5:      {map50:.3f} ({'🥇 Excellent!' if map50 > 0.95 else '🥈 Very Good!' if map50 > 0.90 else '🥉 Good!' if map50 > 0.80 else '⚠️ Needs improvement'})")
            print(f"📈 mAP@0.5:0.95: {map_5095:.3f} ({'🥇 Excellent!' if map_5095 > 0.90 else '🥈 Very Good!' if map_5095 > 0.80 else '🥉 Good!' if map_5095 > 0.70 else '⚠️ Needs improvement'})")
            print(f"🎯 Precision:    {precision:.3f} (How accurate are detections?)")
            print(f"🔍 Recall:       {recall:.3f} (How many objects were found?)")
            
            print("\n💡 Metric Explanations:")
            print("• mAP@0.5: Average precision at 50% overlap threshold")
            print("• mAP@0.5:0.95: Average precision across multiple thresholds (more strict)")
            print("• Precision: Of all detections, how many were correct?")
            print("• Recall: Of all ground truth objects, how many were detected?")
            
            # Performance assessment
            if map50 > 0.95 and map_5095 > 0.90:
                print("\n🎉 OUTSTANDING PERFORMANCE! This model is ready for production deployment.")
            elif map50 > 0.90 and map_5095 > 0.80:
                print("\n✅ EXCELLENT PERFORMANCE! This model will work very well in real-world scenarios.")
            elif map50 > 0.80 and map_5095 > 0.70:
                print("\n👍 GOOD PERFORMANCE! This model should work well for most use cases.")
            else:
                print("\n⚠️ Model performance could be improved. Consider more training data or different parameters.")
                
        else:
            print("❌ Could not extract validation metrics.")
    else:
        print("❌ Best model not found. Training may not have completed successfully.")
        trained_model = None
        best_model_path = None
else:
    print("❌ No results directory found.")
    trained_model = None
    best_model_path = None

## 7. Model Testing

Let's test our trained model on some sample images to see how well it performs in practice!

In [ ]:
# Test the model on sample training images
if 'trained_model' in locals() and trained_model is not None:
    print("🧪 Testing the trained model on sample images...")
    print("This shows how well the model detects and classifies objects!")
    
    # Get some test images (first augmented image of each class)
    train_images_dir = Path('train/images')
    test_images = list(train_images_dir.glob('*_001.jpg'))[:6]
    
    if test_images:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('🔍 Model Predictions on Test Images', fontsize=16, fontweight='bold')
        
        for idx, img_path in enumerate(test_images[:6]):
            row = idx // 3
            col = idx % 3
            
            # Run inference
            results = trained_model(str(img_path), verbose=False, conf=0.5)
            
            # Get the annotated image with predictions
            annotated_img = results[0].plot()
            annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
            
            axes[row, col].imshow(annotated_img_rgb)
            axes[row, col].set_title(f"Test: {img_path.stem}", fontweight='bold')
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        print("\n🎯 What to look for in the predictions:")
        print("• ✅ Bounding boxes around objects (colored rectangles)")
        print("• 🏷️  Class labels (bird, car, dog, cat, motorcycle, truck)")
        print("• 📊 Confidence scores (higher = more confident, >0.5 is good)")
        print("• 🎨 Different colors for different classes")
        print("\n💡 High confidence + correct labels = excellent model performance!")
        
    else:
        print("❌ No test images found.")
else:
    print("❌ Trained model not available. Please complete the training step first.")

In [ ]:
# Function to test model on any image (for future use)
def test_on_custom_image(image_path, confidence_threshold=0.5):
    """Test the trained model on a custom image"""
    if 'trained_model' not in locals() or trained_model is None:
        print("❌ Trained model not available. Please complete the training step first.")
        return
    
    if not Path(image_path).exists():
        print(f"❌ Image not found: {image_path}")
        return
    
    print(f"🔍 Testing model on: {image_path}")
    
    # Run inference
    results = trained_model(image_path, conf=confidence_threshold, verbose=False)
    
    # Display results
    annotated_img = results[0].plot()
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_img_rgb)
    plt.title(f'🎯 Detection Results - {Path(image_path).name}', fontweight='bold', fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()
    
    # Load dataset config to get class names
    with open('dataset.yaml', 'r') as f:
        dataset_config = yaml.safe_load(f)
    
    # Print detection details
    if len(results[0].boxes) > 0:
        print("\n🎯 Detections found:")
        for i, box in enumerate(results[0].boxes):
            class_id = int(box.cls[0])
            confidence = float(box.conf[0])
            class_name = dataset_config['names'][class_id]
            print(f"   {i+1}. {class_name.upper()} (confidence: {confidence:.3f})")
    else:
        print("\n❌ No objects detected above confidence threshold")
        print(f"💡 Try lowering confidence_threshold (current: {confidence_threshold})")

print("📝 Custom testing function created!")
print("💡 Usage example:")
print("   test_on_custom_image('path/to/your/image.jpg', confidence_threshold=0.3)")
print("   (Upload your own images to test the model!)")

## 8. ONNX Conversion for Deployment

Let's convert our trained PyTorch model to ONNX format for efficient deployment on devices like Raspberry Pi. ONNX models are faster and more compatible across different platforms.

In [ ]:
# Convert model to ONNX format optimized for Pi deployment
if 'best_model_path' in locals() and best_model_path and best_model_path.exists():
    print("🚀 Converting trained PyTorch model to ONNX format...")
    print("🥧 Optimizing for Raspberry Pi deployment!")
    
    try:
        # Use built-in YOLO export function (simpler and more reliable than custom script)
        print("\n⚙️ Export Configuration:")
        print("   Format: ONNX")
        print("   Image size: 320x320 (optimized for Pi camera)")
        print("   Precision: FP32 (better Pi compatibility)")
        print("   Simplification: Enabled (faster inference)")
        
        # Export the model
        onnx_path = trained_model.export(
            format='onnx',
            imgsz=320,        # Smaller size for Pi camera (320x240)
            half=False,       # Use FP32 for better Pi compatibility
            simplify=True,    # Simplify the ONNX graph for better performance
            opset=11,         # ONNX opset version (compatible with most systems)
        )
        
        if onnx_path and Path(onnx_path).exists():
            print(f"\n✅ ONNX conversion successful!")
            print(f"📁 ONNX model saved: {onnx_path}")
            
            # Show file sizes
            pytorch_size = best_model_path.stat().st_size / (1024*1024)
            onnx_size = Path(onnx_path).stat().st_size / (1024*1024)
            print(f"📊 File size comparison:")
            print(f"   PyTorch (.pt): {pytorch_size:.1f} MB")
            print(f"   ONNX (.onnx):  {onnx_size:.1f} MB")
            
            # Test the ONNX model
            try:
                import onnxruntime as ort
                session = ort.InferenceSession(onnx_path)
                inputs = session.get_inputs()
                outputs = session.get_outputs()
                
                print(f"\n🔍 ONNX Model Verification:")
                print(f"   ✅ Model loads successfully")
                print(f"   📐 Input shape: {inputs[0].shape}")
                print(f"   📊 Output shape: {outputs[0].shape}")
                print(f"   🖥️  Available providers: {ort.get_available_providers()}")
                
                print(f"\n🎉 Model ready for Raspberry Pi deployment!")
                
            except ImportError:
                print(f"\n⚠️  ONNX Runtime not available for verification")
                print(f"   Model conversion completed, but couldn't verify")
                print(f"   Install with: pip install onnxruntime")
                
        else:
            print("❌ ONNX conversion failed - file not created")
            onnx_path = None
            
    except Exception as e:
        print(f"❌ ONNX conversion failed: {e}")
        print(f"💡 The PyTorch model still works perfectly for most deployments")
        onnx_path = None
        
else:
    print("❌ No trained model found. Please complete training first.")
    onnx_path = None

# Deployment instructions
print(f"\n" + "="*60)
print(f"🥧 RASPBERRY PI DEPLOYMENT INSTRUCTIONS")
print(f"="*60)
print(f"""📋 Quick setup steps:

1. 📥 Download your model files (next cell)
2. 📤 Copy to your Raspberry Pi:
   scp best.pt pi@your-pi-ip:~/
   
3. 🔧 Install dependencies on Pi:
   pip install ultralytics opencv-python
   
4. 🐍 Python inference code:
   ```python
   from ultralytics import YOLO
   model = YOLO('best.pt')
   results = model('image.jpg')
   results[0].show()
   ```

5. 📹 For live camera detection:
   Use the pi_camera_inference.py script included in the repo!
   
🎯 Classes: bird, dog, cat, motorcycle, car, truck
⚡ Expected Pi performance: 2-5 FPS (great for real-time detection!)
""")
print(f"="*60)

## 9. Download Your Trained Models

🎉 **Congratulations!** Your YOLO model is trained and ready for deployment. Let's download the model files so you can use them on your Raspberry Pi or other devices.

In [ ]:
# Download trained models (Colab only)
try:
    import google.colab
    from google.colab import files
    
    print("📥 Preparing model downloads for local use...")
    print("🎯 These files are ready for Raspberry Pi deployment!")
    
    download_count = 0
    
    # Download the best PyTorch model
    if 'best_model_path' in locals() and best_model_path and best_model_path.exists():
        print(f"\n📦 Downloading: {best_model_path.name}")
        print(f"   Size: {best_model_path.stat().st_size / (1024*1024):.1f} MB")
        print(f"   Use: Primary model for inference")
        files.download(str(best_model_path))
        download_count += 1
        
        # Also download the last model (in case best model has issues)
        last_model_path = best_model_path.parent / 'last.pt'
        if last_model_path.exists():
            print(f"\n📦 Downloading: {last_model_path.name} (backup)")
            print(f"   Size: {last_model_path.stat().st_size / (1024*1024):.1f} MB")
            print(f"   Use: Alternative if best.pt has issues")
            files.download(str(last_model_path))
            download_count += 1
    
    # Download ONNX model if it exists
    if 'onnx_path' in locals() and onnx_path and Path(onnx_path).exists():
        print(f"\n📦 Downloading: {Path(onnx_path).name}")
        print(f"   Size: {Path(onnx_path).stat().st_size / (1024*1024):.1f} MB")
        print(f"   Use: Optimized for faster inference")
        files.download(str(onnx_path))
        download_count += 1
    
    # Download dataset config
    dataset_config_path = Path('dataset.yaml')
    if dataset_config_path.exists():
        print(f"\n📦 Downloading: {dataset_config_path.name}")
        print(f"   Use: Contains class names and dataset info")
        files.download(str(dataset_config_path))
        download_count += 1
    
    # Download Pi inference script
    pi_script_path = Path('pi_camera_inference.py')
    if pi_script_path.exists():
        print(f"\n📦 Downloading: {pi_script_path.name}")
        print(f"   Use: Ready-to-run Pi camera detection script")
        files.download(str(pi_script_path))
        download_count += 1
    
    if download_count > 0:
        print(f"\n✅ Downloaded {download_count} files to your browser's Downloads folder!")
        print(f"🚀 Your YOLO model is ready for deployment!")
        
        print(f"\n🎯 Quick Start on Raspberry Pi:")
        print(f"1. Copy best.pt to your Pi")
        print(f"2. Install: pip install ultralytics opencv-python")
        print(f"3. Run: python pi_camera_inference.py")
        print(f"4. Point camera at objects and watch the magic! 🪄")
        
    else:
        print("❌ No model files found to download.")
        print("🔍 Make sure training completed successfully.")
        
except ImportError:
    print("💻 Local environment detected - no downloads needed")
    print("🗂️  Your trained models are saved in:")
    if 'results_dir' in locals() and results_dir:
        print(f"   📁 {results_dir}/weights/")
        print(f"   📄 best.pt (main model)")
        print(f"   📄 last.pt (backup model)")
        if 'onnx_path' in locals() and onnx_path:
            print(f"   📄 {Path(onnx_path).name} (ONNX version)")
    else:
        print("   🔍 Check runs/detect/drone_detection*/weights/")

## 🎉 Congratulations! Mission Accomplished!

You've successfully completed the **DEXI YOLO Training Pipeline** in Google Colab! 🚀

### ✅ What You've Accomplished:

1. **🌟 Environment Setup**: Automatically cloned repo and configured Colab with GPU acceleration
2. **📊 Dataset Exploration**: Examined your 6 base images for training
3. **🔄 Data Augmentation**: Generated 900 diverse training images from just 6 originals
4. **🤖 YOLO Training**: Trained a custom YOLOv8 nano model with excellent accuracy
5. **📈 Results Analysis**: Visualized training curves, confusion matrix, and validation results
6. **🧪 Model Testing**: Verified predictions on test images
7. **⚡ ONNX Conversion**: Optimized model for fast Raspberry Pi deployment
8. **📥 Model Download**: Downloaded all files needed for deployment

### 🏆 Your Model Performance:
- **🎯 Detection Classes**: Bird, Dog, Cat, Motorcycle, Car, Truck
- **📊 Expected Accuracy**: 99%+ mAP@0.5 (exceptional!)
- **⚡ Model Size**: ~6MB (perfect for edge devices)
- **🥧 Pi Performance**: 2-5 FPS real-time detection

### 🚀 Next Steps - Deploy Your Model:

#### **Raspberry Pi Deployment:**
```bash
# 1. Copy your model to Pi
scp best.pt pi@your-pi-ip:~/

# 2. Install dependencies
pip install ultralytics opencv-python

# 3. Run live detection
python pi_camera_inference.py
```

#### **Python Inference (Any Device):**
```python
from ultralytics import YOLO

# Load your trained model
model = YOLO('best.pt')

# Run detection
results = model('your_image.jpg')
results[0].show()  # Display results
```

### 🌟 Key Achievements:
- ⚡ **Lightning Fast**: Completed full training pipeline in ~20 minutes
- 🎯 **Highly Accurate**: State-of-the-art object detection performance  
- 📱 **Edge Ready**: Optimized for mobile and IoT deployment
- 🆓 **Cost Effective**: Trained using free Google Colab resources
- 🔄 **Reproducible**: Complete pipeline from data to deployment

### 📚 Learn More:
- [Ultralytics YOLOv8 Documentation](https://docs.ultralytics.com/)
- [YOLO Model Deployment Guide](https://docs.ultralytics.com/modes/export/)
- [Advanced Training Techniques](https://docs.ultralytics.com/modes/train/)
- [DroneBlocks DEXI Platform](https://droneblocks.io/)

---

**🎯 Happy detecting with your custom YOLO model! 🤖✨**

*Built with ❤️ by DroneBlocks for the DEXI platform*

In [ ]:
# Final summary and success message
print("🎊" + "="*58 + "🎊")
print("🎉         YOLO TRAINING COMPLETED SUCCESSFULLY!         🎉")
print("🎊" + "="*58 + "🎊")
print()
print("🏆 Your custom YOLO model is now trained and ready to deploy!")
print()
print("📊 Training Summary:")
if 'training_duration' in locals():
    print(f"   ⏱️  Training time: {training_duration/60:.1f} minutes")
if 'best_model_path' in locals() and best_model_path:
    print(f"   💾 Model size: {best_model_path.stat().st_size / (1024*1024):.1f} MB")
print(f"   🎯 Classes: bird, dog, cat, motorcycle, car, truck")
print(f"   🚀 Device used: {device.upper()}")
print()
print("🎯 Quick deployment test:")
print("```python")
print("from ultralytics import YOLO")
print("model = YOLO('best.pt')")
print("results = model('your_image.jpg')")
print("results[0].show()")
print("```")
print()
print("🥧 Ready for Raspberry Pi drone detection!")
print("🌟 Built with Google Colab + YOLOv8 + DroneBlocks DEXI")
print()
print("🎊" + "="*58 + "🎊")